In [ ]:
import os

os.chdir("/home/bennour/Pywd/")

#### Prepare data for process 

In [2]:
import numpy as np
import pandas as pd

In [3]:
# producer's data details
data_1 = [
    ["Nig", 24, 75000, 999999999],
    ["Gab", 25, 60000, 200000],
    ["Ven", 22.3, 50000, 999999999],
    ["Mex", 24.2, 50000, 250000],
]
prod_data = pd.DataFrame(data_1, columns=["producer", "price", "MIN", "MAX"])

In [4]:
# products per barrel for each producer
data_2 = [
    ["Nig", 0.22, 0.2, 0.05, 0.53, 4],
    ["Gab", 0.18, 0.22, 0.12, 0.48, 4.25],
    ["Ven", 0.25, 0.05, 0.22, 0.48, 4.15],
    ["Mex", 0.2, 0.26, 0.14, 0.4, 4.5],
]

prod_barrel = pd.DataFrame(
    data_2,
    columns=[
        "producer",
        "unleaded_gasoline",
        "fuel_oil",
        "naphta",
        "by_products",
        "refining_cost",
    ],
)

In [5]:
# demand data for each product
data_3 = [
    ["unleaded_gasoline", 200000, 200],
    ["fuel_oil", 100000, 125],
    ["naphta", 40000, 310],
    ["by_products", 0, 20],
]

demand_data = pd.DataFrame(data_3, columns=["products", "demand", "price"])

## let's take a look at our data  

In [6]:
prod_data

,producer,price,MIN,MAX
0,Nig,24.0,75000,999999999
1,Gab,25.0,60000,200000
2,Ven,22.3,50000,999999999
3,Mex,24.2,50000,250000


In [7]:
prod_barrel

,producer,unleaded_gasoline,fuel_oil,naphta,by_products,refining_cost
0,Nig,0.22,0.20,0.05,0.53,4.00
1,Gab,0.18,0.22,0.12,0.48,4.25
2,Ven,0.25,0.05,0.22,0.48,4.15
3,Mex,0.20,0.26,0.14,0.40,4.50


In [8]:
demand_data

,products,demand,price
0,unleaded_gasoline,200000,200
1,fuel_oil,100000,125
2,naphta,40000,310
3,by_products,0,20


In [9]:
# create a list of producers
producers = prod_barrel.producer
# create a list of products
products = demand_data.products

In [10]:
# create a dictionary of propotions marginal profit per producer
p_nig = dict(zip(products, prod_barrel.iloc[0, 1:5]))
p_gab = dict(zip(products, prod_barrel.iloc[1, 1:5]))
p_ven = dict(zip(products, prod_barrel.iloc[2, 1:5]))
p_mex = dict(zip(products, prod_barrel.iloc[3, 1:5]))

# create a dictionary of proportions marginal profit per product
p_gas = dict(zip(producers, prod_barrel.iloc[0, 1:5]))
p_oil = dict(zip(producers, prod_barrel.iloc[1, 1:5]))
p_naf = dict(zip(producers, prod_barrel.iloc[2, 1:5]))
p_byp = dict(zip(producers, prod_barrel.iloc[3, 1:5]))

In [11]:
from pulp import *

Our objective is to maximize the profit of the refinery by optimizing the number of barrels of each producer. 
let's define 

$i \in I$ where $I = \{1, 2, 3, 4\}$ respectively denoting Nig, Gab, Ven, Mex.

$j \in J$ where $J = \{1, 2, 3, 4\}$ respectively denoting Gasoline, fuel, naphta, and other by products

Let's define the profit function, on which we will build our objective function and solve the problem:

#### *parameters:*
The refeniery's revenue is the ammount of refined products barrels sold, this depends on the proportion of each product by the producer
Let's define: 

$P_{ij}$ : proportion of product $j$ if the barrel is from producer $i$.

$SP_j$ : selling price of a barrel of product $j$.

$RC_i$ : refining cost if the barrel is from producer $i$.

$PP_i$ : purchase price if the barrel is from producer $i$.

$m_i$ : the minimun number of barrels to purchase from $i$.

$M_i$ : the maximum number of barrels to purchase from $i$.

$D_j$ : the minimum demand of product $j$.

#### *decicion variable*

$x_i$ :The number of barrels from producer $i$

### Profit:
The money that we are hoping to maximize is coming from the selling of the refined product in barrels.
This ammount sold is depending on the number of barrels from each producer and the proportion included of each product to refine times the price:

$\sum_{i \in I}\sum_{j \in J}x_iP_{ij}SP_j$

### Cost:

The cost that the refinery takes is dependent on:
the purchase price of the producer $i$ barrel 
the refining cost of each of barrel per producer.

$\sum_{i \in I} x_i(PP_i + RC_i)$

this gives us that our objective function is: 

$Max_x \sum_{i \in I}\sum_{j \in J}x_iP_{ij}SP_j - x_i(PP_i + RC_i)$

In [12]:
# define the problem
prob = LpProblem("refinery", LpMaximize)

In [13]:
X = ['Nig','Gab', 'Ven', 'Mex']
PP = dict(zip(producers, prod_data.price))
RC = dict(zip(producers, prod_barrel.refining_cost))
SP = dict(zip(products, demand_data.price))
pm = dict(zip(producers, prod_data.MIN))
pM = dict(zip(producers, prod_data.MAX))
D = dict(zip(products, demand_data.demand))
C = 900000
P = {
    "Nig": p_nig,
    "Gab": p_gab,
    "Ven": p_ven,
    "Mex": p_mex,
}

In [14]:
# define the variable
x = LpVariable.dicts(name="x",
                     lowBound = 0,
                     indexs = X,
                     cat = "Integer")

In [15]:
# encode obj function
prob += lpSum([[P[i][j] * SP[j] * x[i] - x[i] * (PP[i] + RC[i]) for j in products]for i in producers])

In [16]:
# encode constraints
prob += lpSum(x) <= C
for i in producers:
    prob += x[i] <= pM[i]
for i in producers:
    prob += x[i] >= pm[i]
for j in products:
        prob += lpSum([P[i][j]*x[i] for i in producers]) >= D[j]

In [17]:
# the problem formulation
prob

refinery:
MAXIMIZE
-6.700000000000003*x_Gab + 9.100000000000009*x_Mex + -16.9*x_Nig + 28.249999999999993*x_Ven + 0.0
SUBJECT TO
_C1: x_Gab + x_Mex + x_Nig + x_Ven <= 900000

_C2: x_Nig <= 999999999

_C3: x_Gab <= 200000

_C4: x_Ven <= 999999999

_C5: x_Mex <= 250000

_C6: x_Nig >= 75000

_C7: x_Gab >= 60000

_C8: x_Ven >= 50000

_C9: x_Mex >= 50000

_C10: 0.18 x_Gab + 0.2 x_Mex + 0.22 x_Nig + 0.25 x_Ven >= 200000

_C11: 0.22 x_Gab + 0.26 x_Mex + 0.2 x_Nig + 0.05 x_Ven >= 100000

_C12: 0.12 x_Gab + 0.14 x_Mex + 0.05 x_Nig + 0.22 x_Ven >= 40000

_C13: 0.48 x_Gab + 0.4 x_Mex + 0.53 x_Nig + 0.48 x_Ven >= 0

VARIABLES
0 <= x_Gab Integer
0 <= x_Mex Integer
0 <= x_Nig Integer
0 <= x_Ven Integer

In [23]:
import cplex
prob.solve(CPLEX_PY())

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 10 rows and 0 columns.
MIP Presolve modified 15 coefficients.
Reduced MIP has 3 rows, 4 columns, and 10 nonzeros.
Reduced MIP has 0 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.01 ticks)
Found incumbent of value -5171000.000000 after 0.03 sec. (0.04 ticks)
Tried aggregator 1 time.
Reduced MIP has 3 rows, 4 columns, and 10 nonzeros.
Reduced MIP has 0 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.00 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                     -5171000.0000   2.08042e+07           502.33%
      0     0  

1

In [26]:
pulp.LpStatus[prob.status]
pulp.value(prob.objective)

16882307.7

In [43]:
varsdict = {}
for v in prob.variables():
    varsdict[v.name] = v.varValue

In [44]:
varsdict

{'x_Gab': 60000.0, 'x_Mex': 159762.0, 'x_Nig': 75000.0, 'x_Ven': 605238.0}